In [ ]:
import panel as pn
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

pn.extension('perspective')

The ``Perspective`` pane provides a powerful visualization component for large, real-time datasets built on the [Perspective Project](https://perspective.finos.org/).

The ``Perspective`` pane can

- provide your users with the ability to quickly explore their data via built in pivot and plotting functionality
- visualize large datasets with speed

The ``Perspective`` pane is a very good alternative to the `Tabulator` table.

Check out the [Perspective Examples Gallery](https://perspective.finos.org/examples/) for inspiration.

## Parameters:

For details on other options for customizing the component see the [layout](../../how_to/layout/index.md) and [styling](../../how_to/styling/index.md) how-to guides.

* **``aggregates``** (dict): Aggregation spec, e.g. {x: "distinct count"}
* **``columns``** (list or dict): List of column names to display or a dictionery with column configuration options.
* **``expressions``** (list): List of expressions, e.g. `['"x"+"y"']`
* **``filters``** (list): A list of filters, e.g. `[["x", "<", 3], ["y", "contains", "abc"]]`.
* **``group_by``** (list): List of columns to group by, e.g. `["x", "y"]`
* **``object``** (dict or pd.DataFrame): The plot data declared as a dictionary of arrays or a DataFrame.
* **``selectable``** (bool, default=True): Whether rows are selectable
* **``split_by``** (list):  A list of columns to pivot by. e.g. `["x", "y"]`
* **``sort``** (list): List of sorting specs, e.g. `[["x", "desc"]]`
* **``plugin``** (str): The name of a plugin to display the data. For example `'datagrid'` or `'d3_xy_scatter'`.
* **``plugin_config``** (dict): Configuration for the PerspectiveViewerPlugin 
* **``toggle_config``** (bool): Whether to show the config menu. Default is True.
* **``theme``** (str): The theme of the viewer, available options include `'material'`, `'material-dark'`, `'monokai'`, `'solarized'`, `'solarized-dark'` and `'vaporwave'`

___

The `Perspective` pane renders columns of data specified as a dictionary of lists or arrays and pandas DataFrames:

## Basic Examples

In [ ]:
data = {
    'int': [random.randint(-10, 10) for _ in range(10)],
    'float': [random.uniform(-10, 10) for _ in range(10)],
    'date': [(datetime.now() + timedelta(days=i)).date() for i in range(10)],
    'datetime': [(datetime.now() + timedelta(hours=i)) for i in range(10)],
    'category': ['Category A', 'Category B', 'Category C', 'Category A', 'Category B',
             'Category C', 'Category A', 'Category B', 'Category C', 'Category A'],
}
df = pd.DataFrame(data)

pn.pane.Perspective(df, width=1000)

<div class="alert alert-block alert-info"> <b>NOTE</b> If the Perspective widget is not shown try <em>hard refreshing</em> your browser. </div>

We can also set the various config settings such as choosing the `columns`, `plugin` etc. from Python:

In [ ]:
pn.pane.Perspective(df, columns=["int", "float"], plugin='d3_y_line', width=1000)

## Column Configuration

You can configure columns manually as shown in the image below

![Manual DataGrid Plugin Column Configuration](https://user-images.githubusercontent.com/42288570/244585203-b16947f8-9fff-4a66-a29e-61936ac24ecb.png)

In [ ]:
pn.pane.Perspective(df, width=1000, plugin_config={
    'columns': {
        'X': {'number_fg_mode': 'color', 'neg_fg_color': '#880808', 'pos_fg_color': '#008000', "fixed": 3},
        'Y': { 'number_fg_mode': "disabled", 'fg_gradient': 7.93,  },
    }
})

For more detail about the available options see the [Column Configuration Options Section](#column-configuration-options) below.

## Timezone Adjustments

The underlying Perspective Viewer assumes *non-timezone* aware datetimes are in UTC time. And it displays it in your local time zone.

If you data is not time-zone aware you can make them. My servers time zone is 'cet' and I can make them aware as shown below.

In [ ]:
df_aware = df.copy(deep=True)
df_aware['datetime'] = df_aware['datetime'].dt.tz_localize("cet")
display(df_aware.head(3))
pn.pane.Perspective(df_aware, width=1000)

The datetimes will still be shown to the users in their local timezone though. This is not always what you want though.

## Streaming and Patching

The `Perspective` pane also supports `stream` and `patch` methods allowing us to efficiently update the data

In [ ]:
df_stream = pd.DataFrame(np.random.randn(400, 4), columns=list('ABCD')).cumsum()

stream_perspective = pn.pane.Perspective(
    df_stream, plugin='d3_y_line', columns=['A', 'B', 'C', 'D'], theme='material-dark',
    sizing_mode='stretch_width', height=500, margin=0
)

stream_perspective

Lets start the streaming.

The amount of data to keep in the streaming buffer can be controlled via the `rollover` option:

In [ ]:
rollover = pn.widgets.IntInput(name='Rollover', value=500)

def stream():
    data = df_stream.iloc[-1] + np.random.randn(4)
    stream_perspective.stream(data, rollover.value)

cb = pn.state.add_periodic_callback(stream, 50)

pn.Row(cb.param.period, rollover)

Alternatively we can also `patch` the data:

In [ ]:
mixed_df = pd.DataFrame({'A': np.arange(10), 'B': np.random.rand(10), 'C': [f'foo{i}' for i in range(10)]})

perspective = pn.pane.Perspective(mixed_df, height=500)

perspective

The easiest way to patch the data is by supplying a dictionary as the patch value. The dictionary should have the following structure:

```python
{
    column: [
        (index: int or slice, value),
        ...
    ],
    ...
}
```
    
As an example, below we will patch the 'A' and 'C' columns. On the `'A'` column we will replace the 0th row and on the `'C'` column we replace the first two rows:

In [ ]:
perspective.patch({'A': [(0, 3)], 'C': [(slice(0, 1), 'bar')]})

perspective

Deleting rows can be achieved by streaming the data you want to become visible and setting rollover equal to the row count of new data. Effectively, deleting old rows. Removing specific rows by index in a similar manner as patching is currently not supported.

In [ ]:
data = {'A': np.arange(10)}

perspective = pn.pane.Perspective(data, height=500)

perspective

In [ ]:
smaller_data = {'A': np.arange(5)}

perspective.stream(smaller_data, rollover=5)

## Controls

The `Perspective` pane exposes a number of options which can be changed from both Python and Javascript try out the effect of these parameters interactively:

In [ ]:
pn.Row(perspective.controls(jslink=True), perspective)

## Column Configuration Options

The column configurations options of the underlying Perspective Viewer are not well documented. You can found some in the [Perspective Examples Gallery](https://perspective.finos.org/examples/). They can all be reversed engineered from the [Perspective GitHub repository](https://github.com/finos/perspective).

For example for the `DataGrid` plugin the options for configuring a *number* column are defined in [number_column_style.rs](https://github.com/finos/perspective/blob/master/rust/perspective-viewer/src/rust/config/number_column_style.rs).

Below we list some of the most useful options we have been able to find.

### DataGrid

#### Number Column

- Precision:
  - `fixed`
- Foreground:
  - `number_fg_mode` ('disabled', 'color', 'bar')
  - `pos_fg_color` (hex string), `neg_fg_color` (hex string), `fg_gradient` (float)
- Background:
  - `number_bg_mode` ('disabled', 'color', 'gradient', 'pulse')
  - `pos_bg_color` (str), `neg_bg_color` (str), `bg_gradient` (float)
  
#### String Column